## Import Libraries

This notebook uses scipy for fitting and for peak-finding

In [ ]:
from scipy import optimize, signal, io
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
## constants
frame_rate = 30 # Hz

## Define Fitting Functions

These functions will attempt to fit a sine wave to the x and y components of the alsomitra flight path. This requires a reasonable estimate for the amplitude and frequency of the sine wave

In [ ]:
def generic_sin_wave(x, a, b, c, d):
    """
    a: amplitude
    b: period
    c: phase
    d: offset
    y = a*sin(b*x + c) + d
    """
    return a*np.sin(b*x + c) + d

def generic_straight_line(x, a, b):
    """
    a: slope
    b: intercept
    y = a*x + b
    """
    return a*x + b

def guess_amplitude(y):
    """
    Guess the amplitude of the sin wave
    """
    return (np.max(y) - np.min(y))/2

def guess_period(x, y, width=None):
    """
    Guess the period of the sin wave
    """
    peaks, properties = signal.find_peaks(y, width=width)
    print(f"{peaks=}")
    if np.size(peaks) >= 2:
        mean_peak_dist = int(np.mean(np.diff(peaks)))
        diff_x = x[mean_peak_dist] - x[0]
        f = 2*np.pi/diff_x
    else:
        diff_x = x[-1] - x[0]
        f = 2*np.pi/diff_x
        
    return f
    
def find_initial_params(x, y, width_peaks=None):
    """
    Find the initial parameters for the sin wave fit
    """
    # Some decent initial guesses
    a = guess_amplitude(y)
    b = guess_period(x,y, width=width_peaks)
    print(f"amplitude: {a}")
    print(f"period: {b}")
    return a, b, 1, 1

def fit_sin_wave(f, x, y, yhat, width_peaks=None):
    """
    Fit the sin wave to the data.
    x: x-axis data
    y: y-axis data
    yhat: smoothed y-axis data
    """
    sol, cov = optimize.curve_fit(
        generic_sin_wave, x, y, p0=find_initial_params(x, yhat, width_peaks=width_peaks), maxfev=10000 
    ) # bounds=(-inf, inf)
    print(sol)
    err = np.sqrt(np.diag(cov))
    return sol, cov, err

def fit_straight_line(f, x, y):
    """
    Fit the straight line to the data.
    x: x-axis data
    y: y-axis data
    """
    sol, cov = optimize.curve_fit(
        generic_straight_line, x, y, maxfev=10000 
    ) 
    print(sol)
    err = np.sqrt(np.diag(cov))
    return sol, cov, err

In [ ]:
def read_data():
    """
    Read the data from the file
    """
    with open('x.dat') as f:
        x_lines = f.readlines()
    with open('y.dat') as f:
        y_lines = f.readlines()
    with open('z.dat') as f:
        z_lines = f.readlines()
            
    return x_lines, y_lines, z_lines


In [ ]:
x_lines, y_lines, z_lines = read_data()

In [ ]:
def fit_x_component(case):
    x = np.fromstring(x_lines[case], sep=';')

    t_sample = np.array([i*1/frame_rate for i in range(len(x))])
    y_sample = x
    y_hat = signal.savgol_filter(y_sample, len(y_sample), 7)
    sol, cov, err = fit_sin_wave(generic_sin_wave, t_sample,y_sample, y_hat)
    y_fit = generic_sin_wave(t_sample, sol[0], sol[1], sol[2], sol[3])

    plt.plot(t_sample, y_sample, 'bo' ,label='data')
    #plt.plot(t_sample, y_hat, label='filtered')
    plt.plot(t_sample, y_fit, 'r', label='fit')
    plt.legend(loc="lower left")

    plt.xlabel('time (s)')
    plt.ylabel('x-component (m)')


    plt.show()
    print(f"a: {sol[0]} +- {err[0]}")
    print(f"b: {sol[1]} +- {err[1]}")
    print(f"c: {sol[2]} +- {err[2]}")
    print(f"d: {sol[3]} +- {err[3]}")

def fit_y_component(case):
    y = np.fromstring(y_lines[case], sep=';')

    t_sample = np.array([i*1/frame_rate for i in range(len(y))])
    y_sample = y
    y_hat = signal.savgol_filter(y_sample, len(y_sample), 7)
    sol, cov, err = fit_sin_wave(generic_sin_wave, t_sample,y_sample, y_hat)
    y_fit = generic_sin_wave(t_sample, sol[0], sol[1], sol[2], sol[3])

    plt.plot(t_sample, y_sample, 'bo' ,label='data')
    #plt.plot(t_sample, y_hat, label='filtered')
    plt.plot(t_sample, y_fit, 'r', label='fit')
    plt.legend(loc="lower left")

    plt.xlabel('time (s)')
    plt.ylabel('Y-component (m)')


    plt.show()

    print(f"a: {sol[0]} +- {err[0]}")
    print(f"b: {sol[1]} +- {err[1]}")
    print(f"c: {sol[2]} +- {err[2]}")
    print(f"d: {sol[3]} +- {err[3]}")


def fit_z_component(case):

    z = np.fromstring(z_lines[case], sep=';')

    t_sample = np.array([i*1/frame_rate for i in range(len(z))])
    y_sample = z*1e-3

    sol, cov, err = fit_straight_line(generic_straight_line, t_sample,y_sample)
    y_fit = generic_straight_line(t_sample, sol[0], sol[1])

    plt.plot(t_sample, y_sample, 'bo' ,label='data')
    #plt.plot(t_sample, y_hat, label='filtered')
    plt.plot(t_sample, y_fit, 'r', label='fit')
    plt.legend(loc="lower left")

    plt.xlabel('time (s)')
    plt.ylabel('z-component (m)')
    plt.show()

    print(f"a: {sol[0]} +- {err[0]}")
    print(f"b: {sol[1]} +- {err[1]}")

In [ ]:
fit_z_component(-2)